<a href="https://colab.research.google.com/github/t8101349/LLM-Finetune/blob/main/%E7%94%A8_LoRA_%E5%92%8C_PEFT_%E5%BE%AE%E8%AA%BF_Gemma_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU transformers accelerate bitsandbytes trl peft datasets auto-gptq optimum


In [ ]:
!pip install --upgrade transformers


In [ ]:
import torch
print(torch.cuda.is_available())  # 如果是 False，表示運行在 CPU


True


In [ ]:
!pip install -U "huggingface_hub[cli]"
# 讀取環境變數
hf_token = **your_token

# 登入 Hugging Face CLI
!huggingface-cli login --token $hf_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `kaggle hf2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `kaggle hf2`


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/gemma-2-2b-it"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="cuda:0", attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch

def get_completion5(query, model, tokenizer):
    """
    Generate a response using the fine-tuned model.
    Ensures the query is in the correct precision.
    """
    # Encode input text
    inputs = tokenizer(query, return_tensors="pt")

    # Convert inputs to the same dtype as the model
    #inputs = {k: v.to(model.dtype) for k, v in inputs.items()}  # Ensure dtype matches
    inputs = {k: v.type(torch.int64).to(model.device) for k, v in inputs.items()}

    # Generate response
    with torch.no_grad():
        output = model.generate(**inputs, max_length=100)

    # Decode output
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Kaludi/Customer-Support-Responses", split="train")
dataset

Dataset({
    features: ['query', 'response'],
    num_rows: 74
})

In [ ]:
def generate_prompt(data_point):
    prompt=f"""<start_of_turn>system: You are a support chatbot who helps with user queries chatbot who always responds in the style of a professional.\n
<end_of_turn><start_of_turn>user:{data_point['query']}<end_of_turn>
<start_of_turn>model:
{data_point['response']}
<end_of_turn>"""
    return prompt

In [ ]:
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [ ]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
model=base_model
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
import transformers

from trl import SFTTrainer, SFTConfig



# Define training arguments
training_args = SFTConfig(
    dataset_text_field="prompt",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    max_steps=100,
    learning_rate=2e-4,
    logging_steps=10,
    output_dir="fine_tuning2",
    optim="paged_adamw_8bit",
    save_strategy="epoch",
    report_to="none",
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay = 0.02 # ✅
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Initialize SFTTrainer without `dataset_text_field`
trainer = SFTTrainer(
    model=model.to("cuda:0"),
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=lora_config,
    args=training_args,
    data_collator=data_collator,)

Converting train dataset to ChatML:   0%|          | 0/59 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/59 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/59 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/15 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/15 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.455500
20,0.675200
30,0.481300
40,0.317700
50,0.219400
60,0.155600
70,0.120100
80,0.105800
90,0.094100
100,0.091200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=100, training_loss=0.4715843379497528, metrics={'train_runtime': 410.0777, 'train_samples_per_second': 1.951, 'train_steps_per_second': 0.244, 'total_flos': 681187319801856.0, 'train_loss': 0.4715843379497528})

In [ ]:
new_model = "gemma-finetuned-2"
trainer.model.save_pretrained(new_model)

In [ ]:
import tempfile

# Create a temporary directory for offloading
offload_dir = tempfile.mkdtemp()

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",  # Let Transformers decide the best device
    trust_remote_code=True,  # Trust remote code if necessary
    torch_dtype=torch.float16,
    offload_state_dict=True  # Enable offloading the state dict to CPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
merged_model= PeftModel.from_pretrained(base_model, new_model)

NameError: name 'PeftModel' is not defined

In [ ]:
# Example usage:
query = "My order hasn't arrived yet."
response = get_completion5(query=query, model=merged_model, tokenizer=tokenizer)

print(f"Query: {query}")
print(f"Response: {response}")